In [69]:
from gurobipy import *
import numpy as np

''' 
Modelling a dice game where:
    - Objective: get the highest possible sum of numbers on a simultaneous roll of 3 dice  
    - Before the roll, the player must decide:
        - For first die, either: 
            - 1 to 6 (d1 = 1) or
            - -3 to 9 (d1 = 0)
        - For second die, either: 
            - -4 to 7 (d2 = 1) or
            - -2 to 4 (d2 = 0)
        - For third die, either:
            - -2 to 5 (d3 = 1) or
            - -1 to 3 (d3 = 0)
    - RULE: Cannot choose 1 to 6 die AND -4 to 7 die AND -2 to 5 die (all highest E(x))
'''

# Create model
model = Model("UNIF-MILP")
model.setParam('OutputFlag', 0)

# Create variables
d1 = model.addVar(vtype = GRB.BINARY, name = "d1")
dice1 = list() # A list of dice (size of 2) that decision maker must decide between for the first die 
dice11 = (1, 6) # Represents a die - a tuple signifiying the (lb, ub) of the die
dice12 = (-3, 9)
dice1.append(dice11)
dice1.append(dice12)
d2 = model.addVar(vtype = GRB.BINARY, name = "d2")
dice2 = list()
dice21 = (-4, 7)
dice22 = (-2, 4)
dice2.append(dice21)
dice2.append(dice22)
d3 = model.addVar(vtype = GRB.BINARY, name = "d3")
dice3 = list()
dice31 = (-2, 5)
dice32 = (-1, 3)
dice3.append(dice31)
dice3.append(dice32)
model.update()
allDice = {d1 : dice1, d2: dice2, d3: dice3}

# Returns reparameterized representation of the distribution of a roll as a Gurobi linear expression 
def reparameterizeDice(unif, decision, diceToDecide):
    lb = list()
    ub = list()
    for d in diceToDecide:
        lb.append(d[0])
        ub.append(d[1])
    loc = lb[0]*decision + lb[1]*(1 - decision)
    scale = ub[0]*decision + ub[1]*(1 - decision) - loc
    return loc + scale*unif

# Set objective function - two versions: objn works (so far - needs more testing) but obj doesn't (whhhyyyyyyyy)
n = 10000 # Number of trials 
i = 0
obj = LinExpr() # Adds reparameterizations of all 3 dice n times and then is divided by n (Monte Carlo-ing?)
objn = LinExpr() # Generates n u ~ unif[0,1] for all 3 dice, averages them, and then uses them as constants in obj function
unifs = {d1 : 0.0, d2 : 0.0, d3 : 0.0} # Used to store random variables for obj2 
while i < n:
    for d in allDice:
        unif = np.random.uniform(0.0, 1.0)
        unifs[d] = unifs[d] + unif
        obj = obj + reparameterizeDice(unif, d, allDice[d])
    i = i + 1
obj = (1/n)*obj
die = 1
for d in unifs:
    unifs[d] = unifs[d]/n
    objn = objn + reparameterizeDice(unifs[d], d, allDice[d])
    print("Die " + str(die) + " sample average: " + str(unifs[d]))
    die = die + 1
model.setObjective(obj, GRB.MAXIMIZE)

#Set constraints
model.addConstr(d1 + d2 + d3 <= 2)
model.update()

# Optimize model
model.optimize()

# Print Results 
for v in model.getVars():
    print('%s: %g' % (v.varName, v.x))

print('Obj: %g' % model.objVal)

Die 1 sample average: 0.4946934156919074
Die 2 sample average: 0.4983709390282298
Die 3 sample average: 0.5006530107392994
d1: 1
d2: 0
d3: 1
Obj: 5.96826
